In [1]:
import os
from libv3.utils import *
import pandas as pd
import json

In [2]:
#### paths to all files
normal_vpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/normal/varlist_trial2.json'

faulty1_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial1'
faulty2_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial2'
faulty3_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial3'
faulty4_tpath = '/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial4'

faulty1_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial1.json'
faulty2_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial2.json'
faulty3_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial3.json'
faulty4_vpath = '../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial4.json'


In [3]:
#### load varlist for normal trace
normal_to_number = read_json(normal_vpath)

if normal_to_number != False:
    normal_from_number = mapint2var(normal_to_number)

In [4]:
#### load faulty1 trace and varlist
paths_to_varlist = [faulty1_vpath,faulty2_vpath, faulty3_vpath, faulty4_vpath]
paths_to_trace = [faulty1_tpath, faulty2_tpath, faulty3_tpath, faulty4_tpath]



for path_to_varlist, path_to_trace in zip(paths_to_varlist, paths_to_trace):

    ### check if pair of trace and varlist correct
    assert( os.path.basename(path_to_trace).split('_')[1] in path_to_varlist )
    #### check if varlist consistent with normal trace
    check_consistence, _ = is_consistent([normal_vpath, path_to_varlist])

    if check_consistence != False:
        print('varlist is same as normal')
    else:
        print('correcting the trace to work with normal varlist')
        faulty_trace = read_traces(path_to_trace)

        faulty_to_number = read_json(path_to_varlist)
        faulty_from_number = mapint2var(faulty_to_number)

        ### get trace with varibale names
        faulty_vartrace = []
        for stateint, ts in faulty_trace:
            # print(stateint, ts)
            state_var = faulty_from_number[stateint]
            faulty_vartrace += [[state_var, ts]]

        assert (len(faulty_vartrace) == len(faulty_trace))

        ### convert the trace using the normal varlist
        faulty_trace_new = []
        state_var = ''
        state_int = 0
        max_int = max(normal_to_number.values()) ### to increment the normal varlist
        for state_var, ts in faulty_vartrace:
            if state_var in normal_to_number:
                state_int = normal_to_number[state_var]
                faulty_trace_new += [[state_int, ts]]
            else:
                normal_to_number[state_var] = max_int + 1
                max_int += 1
                faulty_trace_new += [[max_int, ts]]

        assert (len(faulty_trace_new) == len(faulty_trace))
        print(len(normal_to_number))
        # print(os.path.basename(path_to_trace).split('_')[1])
        # print(os.path.basename(path_to_trace).split('_')[1] in path_to_varlist)
        print(path_to_varlist)
        print(path_to_trace)
        ### save the corrected trace and varlist
        with open(path_to_trace, 'w') as f:
            json.dump(faulty_trace_new, f)

        with open(path_to_varlist, 'w') as f:
            json.dump(normal_to_number, f)


varlist 1 is consistent with varlist 0
varlist is same as normal
varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist
75
../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial2.json
/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial2
varlist 1 is consistent with varlist 0
varlist is same as normal
varlist 1 is not consistent varlist 0
correcting the trace to work with normal varlist
75
../trace_data/mamba2/single_thread/version_3/faulty_data/varlist_trial4.json
/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial4


In [ ]:
len(faulty_trace_new)

In [ ]:
len(normal_from_number.keys())

In [ ]:
max(normal_to_number.values())